In [1]:
import torch

from campx.thing import Thing
from campx.agent import Agent
from campx.world import TensorWorld

In [2]:
world = TensorWorld(3,3, agent=Agent(1,1))
world.add_thing(Thing(0,0, implicit_reward=1, category="apple")) # good thing
world.add_thing(Thing(1,2, implicit_reward=-1, category="snake")) # bad thing
world.add_thing(Thing(1,0, implicit_reward=-1, category="snake")) # bad thing

In [3]:
world.start()

In [4]:
data = world.render()
data


    1     0     0
    0     0     0
    0     0     0
    0     0     0
    1     0     1
    0     0     0
    0     0     0
    0     1     0
    0     0     0
[torch.FloatTensor of size 9x3]

In [5]:
pred = torch.FloatTensor([0,1,0,0,0])
world.update(pred)

In [6]:
world.reward()

-1.0